# Librerías a instalar que no vienen por default
Correrlos en cmd para instalarlos<br>  pip install BeautifulSoup4<br>  pip install requests<br>  pip install selenium<br>

Hay que descargar el web driver de tu navegador

# Importar librerías

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

# Código

## Funciones

In [ ]:
def check_exists_by_xpath(webdriver, xpath):
    """
        Regresa True si existe el elemento con el xpath dado, sino False
    """
    try:
        webdriver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


def selectOpc(driver,xpathDiv1, xpathDivOpc, pos):
    """
        Selecciona la opción localizada en la posición pos de xpathDivOpc
        driver: driver del navegador
        xpathDiv1:
    """
    #Damos click para que aparezca el div de opciones
    driver.find_element_by_xpath(xpathDiv1).click() 
    #Div de opciones
    opcs = driver.find_element_by_xpath(xpathDivOpc).find_elements_by_tag_name("div") 
    #Damos clic a la opción
    opcs[pos].click()


def getSizeByTag(driver, xpath, tag):
    """
        Dada un xpath busca todos los elementos con el tag dado y regresa el tamaño
    """
    return len(driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag))


def doClick(driver, xpath):
    driver.find_element_by_xpath(xpath).click()


def getElementsByTag(driver, xpath, tag):
    return driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag)


def checkTable(driver, oneTable):
    rowsPath = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody'
    btnCerrarVisor = '//*[@id="modalPDF"]/div/div/div[3]/button'    
    visorPDF = '//*[@id="ContentPlaceHolder1_framePDF"]'
    rows = getElementsByTag(driver, rowsPath, "tr")                    
    if oneTable:
        nr = len(rows) 
    else:
        nr = len(rows)-1
    print(nr)
    for j in range(nr):
        rows = getElementsByTag(driver, rowsPath, "tr")

        cols = rows[j].find_elements_by_tag_name("td")
        nCol = len(cols)
        
        for y in range(nCol-1):
            print(cols[y].get_attribute("innerHTML"))
                                
        #Abrir pdf para obtener link            
        cols[nCol-1].click()        
        #Esperamos a que cargue
        time.sleep(3.5)        
                           
        print(driver.find_element_by_xpath(visorPDF).get_attribute('src'))

        #Cerrar el visor
        doClick(driver, btnCerrarVisor)                        

        #Boton descargar
        #//*[@id="download"]    
    time.sleep(3)


## Web scraping

In [ ]:
#Variar Tipo de Juzgado, Delitos-Modalidades,  Criterios , Tipo de resolución
#Checar el comentario del primer for (i1)

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")
divJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[1]'
divOpcJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[2]/div'
divDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[1]'
divOpcDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[2]/div'
divCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[1]'
divOpcCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[2]/div'
divResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[1]'
divOpcResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[2]/div'
btnBuscar = '//*[@id="ContentPlaceHolder1_BtnBuscar"]'
tabla = '//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'
indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'

# Tipo de juzgado ----------------------------------------------------
selectOpc(driver,divJuzgado,divOpcJuzgado, 0)
t1 = getSizeByTag(driver, divOpcJuzgado, "div")

#Si ya es la versión final hay que cambiar el for para que empiece con 1.

#Empieza en 2 para probar
for i1 in range(2,t1):                                     
    selectOpc(driver,divJuzgado,divOpcJuzgado, i1)   

    #Delitos - Modalidades  ----------------------------------------------  
    selectOpc(driver, divDelitos, divOpcDelitos, 0)        
    t2 = getSizeByTag(driver, divOpcDelitos,"div")

    #Las primeras 7 son Ver todos y opciones vacías
    for i2 in range(7,t2):                           
        selectOpc(driver, divDelitos, divOpcDelitos, i2)        

        #Criterios Específicos ----------------------------------------------      
        selectOpc(driver, divCriterios ,divOpcCriterios , 0)                              
        t3 = getSizeByTag(driver, divOpcCriterios,"div")

        for i3 in range(t3):             
            selectOpc(driver, divCriterios ,divOpcCriterios , i3)               

            #Tipo resolución --------------------------------------------------                    
            selectOpc(driver, divResolucion, divOpcResolucion, 0)
            t4 = getSizeByTag(driver, divOpcResolucion,"div")

            for i4 in range(1, t4):                      
                selectOpc(driver, divResolucion, divOpcResolucion, i4)

                #Hacemos consulta -----------------------------------------------
                doClick(driver, btnBuscar)
                time.sleep(3)
                #Pendiente Checar si ya dio el resultado en lugar de poner 3 segundos    
                                
                if check_exists_by_xpath(driver, tabla):
                    #Hay tabla

                    if check_exists_by_xpath(driver, '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'):        
                        first = True
                        while True:
                            ind = getElementsByTag(driver,indexList, "a")
                            if ind[-1].get_attribute("text") != "...":
                                break
                            #First sirve para saber en que index empezar, porque solo las primeras 10 son de las forma [1,..., 10, "..."]
                            #En ese caso empezamos en 0.
                            #Las demás serán  ["...", n, n+1, ..., n+9, "..."] entonces hay que empezar en 1, sino se regresa a las anteriores
                            if first:
                                i = 0
                                first=False
                            else:
                                i=1
                            start = True
                            while ind[-1].get_attribute("text") == "...":  
                                checkTable(driver,False) 
                                #Por alguna razón después de recorrer la tabla cambia el html y el encabezado se hace parte
                                # de las filas y deja de existir el thead
                                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr'                                
                                ind = getElementsByTag(driver,indexList, "a")
                                ind[i].click()           
                                if(ind[i].get_attribute("text") == "..."):
                                    break                                
                                i+=1
                                #Este es opcional, hay que buscar una manera de saber cuando ya cargó la página                
                                time.sleep(30)
                                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  
                                #Después de que cambiamos de página el html regresa a lo normal y el thead ya existe de nuevo
                                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'
                                ind = getElementsByTag(driver,indexList, "a")              
                            time.sleep(30)
                        #Ya estamos en la última página de sentencias
                        checkTable(driver,False)                                                        
                    else:
                        checkTable(driver,True)

                                            


## Prueba para cuando hay múltiples páginas de sentencias

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")
indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'            

print("Entra")
if check_exists_by_xpath(driver,'//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'):
    #Hay tabla    
    if check_exists_by_xpath(driver, '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'):        
        first = True
        while True:
            ind = getElementsByTag(driver,indexList, "a")
            if ind[-1].get_attribute("text") != "...":
                break
            #First sirve para saber en que index empezar, porque solo las primeras 10 son de las forma [1,..., 10, "..."]
            #En ese caso empezamos en 0.
            #Las demás serán  ["...", n, n+1, ..., n+9, "..."] entonces hay que empezar en 1, sino se regresa a las anteriores
            if first:
                i = 0
                first=False
            else:
                i=1
            start = True
            while ind[-1].get_attribute("text") == "...":  
                checkTable(driver,False) 
                #Por alguna razón después de recorrer la tabla cambia el html y el encabezado se hace parte
                # de las filas y deja de existir el thead
                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr'                                
                ind = getElementsByTag(driver,indexList, "a")
                ind[i].click()           
                if(ind[i].get_attribute("text") == "..."):
                    break                                
                i+=1
                #Este es opcional, hay que buscar una manera de saber cuando ya cargó la página                
                time.sleep(30)
                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  
                #Después de que cambiamos de página el html regresa a lo normal y el thead ya existe de nuevo
                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'
                ind = getElementsByTag(driver,indexList, "a")              
            time.sleep(30)
        #Ya estamos en la última página de sentencias
        checkTable(driver,False)
        
                
    else:
        checkTable(driver,True)
